# GA TSP Projekt – SAPI / TSM
Travelling Salesman Problem (TSP) genetikus algoritmussal, valós koordinátákkal, swap + inversion mutációval.

Ez a notebook projekt-sablon, amelyet ki kell töltened. Ahol `TODO` jelölést látsz, ott neked kell megírni a kódot.

## 1. Városok és koordináták (valós adatok)

Feladat:

- Definiálj legalább 15–25 várost valós vagy mesterséges, de reálisnak tűnő koordinátákkal.
- Tárold őket Python-listában vagy más megfelelő szerkezetben.
- Később ebből fogunk távolságmátrixot építeni.

In [ ]:
# TODO: definiáld a városokat és koordinátáikat
cities = [
    # {"name": "Budapest", "x": 47.4979, "y": 19.0402},
    # {"name": "Győr", "x": 47.6875, "y": 17.6504},
    # ...
]

len(cities), cities[:3]  # ellenőrzésképp

## 2. Távolságfüggvény és távolságmátrix

Feladatok:

- Írj függvényt, amely két város között távolságot számol (pl. euklideszi).
- Írj függvényt, amely a `cities` listából NxN távolságmátrixot épít.

In [ ]:
import math

# TODO: távolság két város között
def distance(city_a, city_b):
    """Számold ki a (city_a, city_b) távolságát (pl. euklideszi)."""
    # TODO
    raise NotImplementedError


# TODO: távolságmátrix építése
def build_distance_matrix(cities):
    """Adja vissza az NxN-es távolságmátrixot a megadott városlistára."""
    # TODO
    raise NotImplementedError


dist_matrix = build_distance_matrix(cities)
len(dist_matrix), len(dist_matrix[0])  # ellenőrzés

## 3. Tour hossza és fitness függvény

Feladatok:

- Írj függvényt, amely egy permutáció (tour) hosszát számolja ki a távolságmátrix alapján.
- Definiálj fitness függvényt (pl. `-tour_length` vagy `1.0 / tour_length`).

In [ ]:
# TODO: tour hossza
def tour_length(tour, dist_matrix):
    """Számold ki a tour hosszát, feltételezve, hogy visszatér az első városba."""
    # TODO
    raise NotImplementedError


# TODO: fitness függvény
def fitness(tour, dist_matrix):
    """Alakítsd a tour hosszát olyan értékké, amit maximalizálni szeretnénk."""
    # pl. return -tour_length(tour, dist_matrix)
    # TODO
    raise NotImplementedError

## 4. Permutációk és kezdeti populáció

Feladatok:

- Írj függvényt, amely véletlen permutációt generál 0..N-1 között.
- Írj függvényt, amely `pop_size` méretű kezdeti populációt hoz létre.

In [ ]:
import random

# TODO: véletlen tour
def random_tour(n_cities):
    """Adj vissza egy véletlen permutációt 0..n_cities-1 között."""
    # TODO
    raise NotImplementedError


# TODO: kezdeti populáció
def init_population(pop_size, n_cities):
    """Populáció: pop_size darab véletlen tour."""
    # TODO
    raise NotImplementedError


pop_size = 50  # állítható paraméter
n_cities = len(cities)

population = init_population(pop_size, n_cities)
len(population), population[0]

## 5. Mutációs operátorok: swap és inversion

Feladatok:

- Implementálj `swap_mutation` függvényt (két pozíció felcserélése).
- Implementálj `inversion_mutation` függvényt (köztes szakasz megfordítása).

In [ ]:
# TODO: swap mutáció
def swap_mutation(tour):
    """Válassz ki két pozíciót és cseréld meg őket egy másolatban."""
    # TODO
    raise NotImplementedError


# TODO: inversion mutáció
def inversion_mutation(tour):
    """Válassz ki két indexet i<j, és fordítsd meg a tour[i:j+1] szakaszt."""
    # TODO
    raise NotImplementedError

## 6. Crossover operátor TSP-re (OX vagy PMX)

Feladat:

- Válassz egy TSP-kompatibilis crossover-t (pl. OX vagy PMX).
- Implementáld úgy, hogy az utódok továbbra is érvényes permutációk legyenek.

In [ ]:
# TODO: OX vagy PMX crossover implementációja
def crossover(parent1, parent2, method="OX"):
    """TSP kompatibilis crossover. Visszaadhat 1 vagy 2 utódot is."""
    # TODO
    raise NotImplementedError

## 7. Szelekciós operátor (rulette / tournament / rang alapú)

Feladat:

- Válassz egy szelekciós módszert (pl. tournament).
- Írj függvényt, amely egy szülőt választ a populációból a fitneszek alapján.

In [ ]:
# TODO: szelekció – pl. tournament selection
def select_parent(population, fitness_values, method="tournament", k=3):
    """Válassz ki egy szülőt a population-ből a fitness_values alapján."""
    # TODO
    raise NotImplementedError

## 8. GA fő ciklus, elitizmus, paraméterek

Feladat:

- Írj `run_ga` függvényt, amely:
  - inicializálja a populációt,
  - minden generációban:
    - kiszámítja a fitnesseket,
    - kiválaszt szülőket,
    - alkalmazza a crossover-t,
    - alkalmazza a mutációt (swap / inversion / kevert),
    - opcionálisan elitizmust.
- Visszaadja a legjobb tour-t, a hozzá tartozó hosszt, és a konvergenciagörbéket.

In [ ]:
# TODO: GA fő ciklus
def run_ga(
    cities,
    pop_size=50,
    n_generations=300,
    crossover_rate=0.9,
    mutation_rate=0.1,
    mutation_mode="mixed",  # "swap", "inversion", "mixed"
    elitism=1
):
    """Futtasd a GA-t TSP-re, és add vissza a legjobb megoldást és statisztikákat."""
    # TODO: implementáció
    raise NotImplementedError


# TODO: paraméterezett futtatás (példa)
# best_tour, best_length, history = run_ga(cities)

## 9. Vizualizáció

Feladatok:

- Ábrázold a legjobb tour-t a síkon (városok pontként, élek vonallal kötve).
- Rajzold ki a konvergenciagörbét (generáció vs. legjobb/átlag fitness vagy tour hossz).

In [ ]:
import matplotlib.pyplot as plt

# TODO: rajzold ki a legjobb tour-t
def plot_tour(cities, tour, title="Best tour"):
    # TODO
    raise NotImplementedError


# TODO: konvergenciagörbe rajzolása
def plot_convergence(history):
    # history: pl. dict {"best": [...], "avg": [...]}
    # TODO
    raise NotImplementedError

## 10. Kísérletek és rövid beszámoló

Feladat:

- Hasonlítsd össze legalább három beállítás eredményét:
  - csak swap mutáció
  - csak inversion mutáció
  - kevert swap + inversion
- Minden beállítást futtasd többször (pl. 10 run), és hasonlítsd össze:
  - átlagos legjobb tour hossz
  - konvergencia viselkedés
- Írj rövid szöveges összefoglalót az eredményekről.

### Rövid elemzés (ide írd a szöveges eredményeket)

- Beállítások:
  - ...
- Eredmények összefoglalása:
  - ...
- Következtetések:
  - ...